<h1><center> Windows function using postgresSQL working functionality comparison with Pandas library </center> </h1>
 > Having good knowledge of SQL and Pandas is essential for Data Scientist.
<br> > This module will teach you how to work with postgreSQL and pandas library. </br>
<br> > Both has pros and cons it will give you idea about when to apply certain function and its technique.
<br> > For demonstration purpose I have used wine quality dataset which you can easily get from Kaggle.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('winequality-red.csv')

In [3]:
df.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5


In [4]:
#Creating a categorical variable with  existing data
mapping = {3 : 'Bad', 
           4 : 'worst', 
           5 : 'drinkable', 
           6 : 'tasty', 
           7 : 'good', 
           8 : 'too good'}
df['quality-cat'] = df['quality'].map(mapping)

In [5]:
# this will work same like sql windows function row_number()
#First it wil group each category and then assign number according to the categories

df['new-col'] = df.groupby(['quality-cat']).cumcount()+1

In [6]:
# we are finding total number of rows in each category

'''Using PostgreSQL
SELECT *,         
   COUNT(*) OVER(PARTITION BY quality-cat) AS ct 
FROM df;'''
df['count'] = df.groupby(['quality-cat'])['residual sugar'].transform('count')

In [7]:
# Here are finding Average value in each category
''' Using PostgreSQL
SELECT *,         
   AVG(residual sugar) OVER(PARTITION BY quality-cat ORDER BY pH ) AS avg 
FROM df;
'''
df['cumsum'] = df.groupby(['quality-cat'])['residual sugar'].transform('cumsum')

In [8]:
# Lead function in each category
# Lead function will supercede the current value if the next value is exhausted then it will print NaN

''' Using PostgreSQL
SELECT *, 
   
   LEAD(salary) OVER(PARTITION BY gender ORDER BY salary) as LEAD
FROM df 
'''
df['lead'] = df.groupby(['quality-cat'])['residual sugar'].transform(lambda x : x.shift(1))

In [9]:
#Lag function in each category
#Lag function will precede the current value if the previou value is not available then it will print NaN

''' Using PostgreSQL
SELECT *, 
   LAG(salary) OVER(PARTITION BY gender ORDER BY salary) as LAG 
FROM df 
'''
df['lag'] = df.groupby(['quality-cat'])['residual sugar'].transform(lambda x : x.shift(-1))

In [10]:
# Ranking in each category

# dense_rank() will assign  rank according to the priority it will not skip any row as contrary to rank()
'''
Using PostgreSQL
SELECT *,         
   DENSE_RANK() OVER(PARTITION BY quality-cat ORDER BY residual sugar) AS Rank 
FROM df
'''

df['Rank'] = df.groupby(['quality-cat'])['residual sugar'].rank('dense',ascending = False)

In [11]:
# counting total number elements by each category

'''
Using PostgreSQL
SELECT quality-cat, 
   COUNT(*) 
FROM df 
GROUP BY quality-cat 
'''
df.groupby(['quality-cat'])['residual sugar'].apply(lambda x: x.size).reset_index() #No of rows by group 

,quality-cat,residual sugar
0,Bad,10
1,drinkable,681
2,good,199
3,tasty,638
4,too good,18
5,worst,53


In [12]:
# Applying quantile function in each category
'''
Using PostgreSQL
 SELECT quality-cat, 
    percentile_disc(0.9) WITHIN GROUP(ORDER BY residual sugar) 
 FROM df 
 GROUP BY quality-cat
'''

df.groupby(['quality-cat']).quantile(0.9).reset_index()

,quality-cat,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,new-col,count,cumsum,lead,lag,Rank
0,Bad,10.52,1.2245,0.507,4.395,0.2067,21.4,48.1,0.999540,3.558,0.653,10.91,3.0,9.1,10.0,24.73,4.540,4.540,7.1
1,drinkable,10.00,0.7750,0.490,3.450,0.1120,32.0,113.0,0.999180,3.500,0.790,11.00,5.0,613.0,681.0,1545.25,3.455,3.455,59.0
2,good,11.60,0.6030,0.630,4.620,0.1002,30.2,60.6,0.999020,3.470,0.880,12.70,7.0,179.2,199.0,499.74,4.630,4.630,44.0
3,tasty,10.96,0.7065,0.520,3.200,0.1053,29.0,74.0,0.999215,3.510,0.850,12.00,6.0,574.3,638.0,1421.63,3.140,3.200,43.0
4,too good,10.88,0.5850,0.593,4.080,0.0809,29.8,77.9,0.997930,3.518,0.878,13.58,8.0,16.3,18.0,43.20,4.240,3.760,11.3
5,worst,10.06,1.0110,0.472,4.380,0.1046,26.0,73.8,0.998080,3.590,0.694,11.40,4.0,47.8,53.0,132.58,4.390,4.260,20.8


In [13]:
df.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality-cat,new-col,count,cumsum,lead,lag,Rank
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,drinkable,1,681,1.9,NaN,2.6,56.0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,drinkable,2,681,4.5,1.9,2.3,46.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,drinkable,3,681,6.8,2.6,1.9,51.0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,tasty,1,638,1.9,NaN,1.8,41.0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,drinkable,4,681,8.7,2.3,1.8,56.0
